## 専門知識労働者
- 保険テクノロジー企業である Insurellm の従業員が使用する、専門知識を持つナレッジ ワーカーである質問応答エージェント。エージェントは正確である必要があり、ソリューションは低コストである必要があります。
- このプロジェクトでは、RAG（検索拡張生成）を使用して、質問/回答アシスタントが高い精度を確保します。ココ、5つめの実装では、RAGのChatインターフェイスからの試行を幾つか修正し動作を確認します。
  - OpenAI → Ollama
  - チャンク数を25に増やす
  - デバッグ用ハンドラを追加
  - EXERCISEで引用付きに変更

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# Langchain、Plotly、およびChromaの輸入

# langchain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# VDBサポート (Chroma)
from langchain_chroma import Chroma

In [3]:
# 価格は重要な要素（と言う建付け）なので、低コストモデルを採用
MODEL = "gpt-4o-mini"

# コレはVDBのChromaのDB名
db_name = "vector_db"

In [4]:
# .envファイルから環境変数をロード
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# langChain のローダーを使用してKBのすべてのサブフォルダ内のすべてのドキュメントを読み取りリスト化
# 余談：メタデータは、DirectoryLoader および TextLoader によって生成され、そこにカスタムの属性、doc_typeを追加している。

folders = glob.glob("knowledge-base/*")

# 一部のユーザーに必要な修正を提供してくれた、コース受講生のCGとJon Rに感謝します。
text_loader_kwargs = {'encoding': 'utf-8'}
# それでもうまくいかない場合は、Windowsユーザーの中には次の行のコメントを解除する必要があるかもしれません。
# text_loader_kwargs={'autodetect_encoding': True}

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs]) # add_metadataを使うよう書き換えられている。

# テキストを200文字の重複部分を持たせた1000文字ごとのチャンク（かたまり）に分割
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# document も chunk も LangChain の Document クラス
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}") # チャンク・リストの長さ
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}") # doc_type

Created a chunk of size 1088, which is longer than the specified 1000


Total number of chunks: 123
Document types found: {'products', 'employees', 'contracts', 'company'}


In [6]:
# 各チャンクに埋込ベクトルを関連付けるVDBに格納
# Chroma は SQLite ベースの人気のOSS VDB

embeddings = OpenAIEmbeddings()

# Hugging Faceのフリーのベクトル埋め込みを使用したい場合
# 次と、embeddings = OpenAIEmbeddings() を交換します：

# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma DataStoreがすでに存在するかどうかを確認してください - もしそうなら、コレクションを削除してゼロから始める

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Chroma VectorStoreを作成してください！

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 123 documents


In [7]:
# ベクターを調査

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 123 vectors with 1,536 dimensions in the vector store


## ベクトルストアの可視化
コードや表示される内容に変更はないので割愛

# LangChainを使用してすべてをまとめる時間です

## GradioのChatインターフェイスを使用して、これを紹介します。
- LLMとのRAGチャットをプロトタイプする迅速かつ簡単な方法
- `Who received the prestigious IIOTY award in 2023?` とでも聞いてみる。
- ポイントは、既定のチャンク数を25に増やすと回答改善する理由をデバッグできた点。

In [8]:
# 舞台裏で送られるものを調査

from langchain_core.callbacks import StdOutCallbackHandler

# OpenAIとの新しいチャットを作成します
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# 代替品 - Ollamaをローカルに使用したい場合は、このラインを除外
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# チャットの会話メモリを設定します
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# RAGに使用されるベクターストアの抽象化。
retriever = vectorstore.as_retriever()

# それをまとめる：GPT 4o-Mini LLM、ベクトルストア、メモリ、デバッグ用ハンドラで会話チェーンをセットアップ
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

# 簡単な質問だが、ココでは対象のチャンクが取得できず回答できない。
query = "Who received the prestigious IIOTY award in 2023?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\ nanswer：", answer)

<>:24: SyntaxWarning: invalid escape sequence '\ '
<>:24: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_33222/3000209947.py:24: SyntaxWarning: invalid escape sequence '\ '
  print("\ nanswer：", answer)
/tmp/ipykernel_33222/3000209947.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
- **2022**: **Satisfactory**  
  Avery focused on rebuilding team dynamics and addressing employee concerns, leading to overall improvement despite a saturated market.  

- **2023**: **Exceeds Expectations**  
  Market leadership was regained with innovative approaches to personalized insurance solutions. Avery is now recognized in industry publications as a leading voice in Insurance Tech innovation.

## Annual Performance History
- **2020:**  
  - Completed onboarding successfully.  
  - Met expectations in delivering project milestones.  
  - Received positive feedback from the team leads.

- **2021:**  
  - Achieved a 95% success rate in proje

In [9]:
# RAGに使用されるベクターストアの抽象化。 Kは、使用するチャンクの数
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# それをまとめる：GPT 4o-Mini LLM、ベクトルストア、メモリ、デバッグ用ハンドラで会話チェーンをセットアップ
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

In [10]:
# 関数のラッピング - 履歴の記憶はconversation_chainにあるため、Gradioのhistoryは使用されていない。
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [11]:
# そしてGradioで：
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7862/: Operation not supported


# 練習問題 - ソース引用を含む回答の生成
https://github.com/ed-donner/llm_engineering/blob/main/week5/community-contributions/day5%20-%20generating%20answers%20with%20citations.ipynb

In [12]:
from openai import OpenAI
from IPython.display import Markdown, display, update_display

openai = OpenAI()

In [13]:
# システム・プロンプト
system_message = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Use the following markdown format to answer the question along with the Source used to generate the answer, add inline citation for each sentence & add end of the answer citations:
'CEO of Insurellm is Avery Lancaster [[1]](Source Link 1). Who is also a co-founder [[2]](Source Link 2)
Citations: (Note: No duplicates allowed in the below list)

[1 - Source Title 1](Link 1)
[2 - Source Title 2](Link 2)
...
[n - Source Title n](Link n)'
 
Example answer: 
'CEO of Insurellm is Avery Lancaster [[1]](knowledge-base\\company\\about.md). Who is also a co-founder [[2]](knowledge-base\\employees\\Avery Lancaster.md)
Citations:

[1 - About Company](knowledge-base\\company\\about.md)
[2 - Avery Lancaster employees](knowledge-base\\employees\\Avery Lancaster.md)'
 
Important Note: Have unique end of the answer citations. Don't give duplicate citation numbers for the same source link, reuse the same citation number if the same source link is referenced multiple times.
"""

x = """
「あなたは質問回答タスクのアシスタントです。
以下のコンテキスト情報を用いて質問に答えてください。
答えがわからない場合は、わからないとだけ答えてください。
回答は最大3文とし、簡潔にまとめてください。
以下のマークダウン形式を用いて、回答の生成に使用した情報源とともに質問に回答し、各文にインライン引用を追加し、回答の末尾に引用を追加してください。
「InsurellmのCEOはAvery Lancasterです[[1]](情報源リンク 1)。彼は共同創設者でもあります[[2]](情報源リンク 2)。」
引用：（注：以下のリストでは重複は許可されていません）

[1 - 情報源タイトル 1](リンク 1)
[2 - 情報源タイトル 2](リンク 2)
...
[n - 情報源タイトル n](リンク n)」

回答例：
「InsurellmのCEOはAvery Lancasterです」 [[1]](knowledge-base\\company\\about.md)。共同創業者でもある[[2]](knowledge-base\\employees\\Avery Lancaster.md)
引用：

[1 - 会社概要](knowledge-base\\company\\about.md)
[2 - Avery Lancaster 従業員](knowledge-base\\employees\\Avery Lancaster.md)

重要：回答の末尾の引用は必ず一意にしてください。同じソースリンクに重複した引用番号を付けないでください。同じソースリンクが複数回参照されている場合は、同じ引用番号を再利用してください。
"""

In [14]:
# チャンクにメタデータを追加
def generate_user_prompt(message):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 25})
    results = retriever.invoke(message)
    doc_chunk_merged = ""
    for doc_chunk in results:        
        source = f"https://github.com/ed-donner/llm_engineering/tree/main/week5/" + doc_chunk.metadata.get("source").replace("\\","/")
        title = doc_chunk.metadata.get("doc_type") + " -> " + source.split('\\')[-1][:-3]
        doc_chunk_merged += f"Content: {doc_chunk.page_content}\n Source title: {title}\n Source link: {source}\n\n"
    return f"Question: {message}\n {doc_chunk_merged}"

In [15]:
# LangChainでretrieverにカスタム・コンテキスト（メタデータ）を追加する処理は面倒なので、素で処理する場合、以下のようになる。
# ・LangChainでretrieverにカスタム・コンテキスト（メタデータ）を追加する処理は調査したがエラーで動作せず
# ・LangChainは、簡単な動作のカスタマイズにも知識が必要になり、また、バージョンアップでの破壊的変更が問題だ。
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": generate_user_prompt(message)}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True, seed=3, max_tokens=1000)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

### テストコードで `Please explain what Insurellm is in a couple of sentences` と聞いてみる

In [16]:
#Testing the Answer generation - 1
user_prompt = "Please explain what Insurellm is in a couple of sentences"

display_handle = display(Markdown(""), display_id=True)
for chunk in chat(user_prompt, []):
    update_display(Markdown(chunk), display_id=display_handle.display_id)

Insurellm is an insurance tech startup founded by Avery Lancaster in 2015, aiming to innovate the insurance industry with its products. It offers various software solutions, including Markellm, a marketplace connecting consumers with insurance providers, and has expanded to 200 employees and over 300 clients worldwide [[1]](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/about.md) [[2]](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/overview.md). 

Citations:

[1 - About Company](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/about.md)  
[2 - Overview of Insurellm](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/overview.md)

### テストコードで `Please explain in short on what products are available in Insurellm` と聞いてみる

In [17]:
#Testing the Answer generation - 2
user_prompt = "Please explain in short on what products are available in Insurellm"

display_handle = display(Markdown(""), display_id=True)
for chunk in chat(user_prompt, []):
    update_display(Markdown(chunk), display_id=display_handle.display_id)

Insurellm offers four key software products: Carllm, a portal for auto insurance companies; Homellm, designed for home insurance providers; Rellm, an enterprise platform for the reinsurance sector; and Marketllm, a marketplace connecting consumers with insurance providers [[1]](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/overview.md). Each product leverages advanced technologies to enhance user experience, streamline processes, and provide tailored solutions [[2]](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/products/Rellm.md). These offerings cater to both B2B and B2C customers, aiming to disrupt traditional insurance practices [[3]](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/about.md).

Citations:
[1 - Overview of Insurellm](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/overview.md)  
[2 - Rellm Product Summary](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/products/Rellm.md)  
[3 - About Insurellm](https://github.com/ed-donner/llm_engineering/tree/main/week5/knowledge-base/company/about.md)

### Gradio の Chatで `Who received the prestigious IIOTY award in 2023?` と聞いてみる

In [18]:
#Launch Gradio
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
